In [ ]:
import pandas as pd
import spacy
import da_core_news_sm
import re

df_sentences = pd.read_csv('data/sentences.csv')


In [ ]:
# text list from the worst
# texts = worst.sentence.tolist()
# tokens = nlp(texts[2])
# tokens = [token.text for token in tokens]
# #tokens
# bert.predict(tokens, IOBformat=False)

# extract named entities from the worst sentences
# import spacy
# import da_core_news_sm
# nlp = da_core_news_sm.load()

nlp = da_core_news_sm.load()

def get_named_entities(text):
    """Extract named entities from text"""
    # trim for whitespace, Bert does not like trailing whitespace. At all.
    text = re.sub(r'\x95', '', text)
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)

    doc = nlp(text)
    tokens = [token.text for token in doc]
    preds = bert.predict(tokens, IOBformat=False)
    return [(ent['text'], ent['type']) for ent in preds['entities']]

In [ ]:
collector =[]# get named entities for each sentence
bad_sentences=[]
for index, row in df_sentences.iterrows():
    text = row['sentence'].strip()
    try:
        ents = get_named_entities(text)
        for ent in ents:
            collector.append({'sentence_id': index, 'text': ent[0], 'type': ent[1]})
    except Exception as e:
        bad_sentences.append({'sentence_id': index, 'text': text})


In [ ]:
df_ents = pd.DataFrame(collector)
df_ents.to_csv('NER_entities.csv')


In [ ]:
# group and count per text and type
grouped = df_ents.groupby(['text', 'type']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
# export to csv

# rename PER to person in values
grouped['type'] = grouped['type'].replace('PER', 'person')
# loc to sted
grouped['type'] = grouped['type'].replace('LOC', 'sted')
# org to organisation
grouped['type'] = grouped['type'].replace('ORG', 'organisation')

grouped.to_csv('NER_entities_grouped.csv')


